In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import plot_model
from keras.models import Model
from keras import backend as K


from keras.utils import multi_gpu_model
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import tensorflow as tf

import gc

import time


%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True

hdf5_file = h5py.File(hdf5_path, "r")

if subtract_mean:
    mm = hdf5_file["train_mean"][...,0]
    mm = mm[np.newaxis, ...]

data_num = hdf5_file["train_flow"].shape[0]
    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

x_train = hdf5_file["train_flow"][...,0]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:,...]
hdf5_file.close()
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")


x_test = hdf5_file["IoT_flow"][...,0]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:,...]

hdf5_file.close()

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

#define number of packets used to train for each device
packets = 200
x_train = x_train[:,:packets,:]
x_test = x_test[:,:packets,:]

input_shape = (x_train.shape[1], x_train.shape[2])
    
    
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()

for i in range (1,50):
    size = 64
    nb_filters = 32
    num_classes = 2
    epochs=30
    activations= "tanh"
    maxlen=packets
    dropout = 0.24295335733172563
    batch_size = 512
    pool_size = 3
    lr = 0.0012168203727788032
    adam = keras.optimizers.Adam(lr=lr)

    optim = adam

    layers = 3
    model = Sequential()
    model.add(Conv1D(filters=nb_filters,kernel_size=size,input_shape=(input_shape),padding="valid",activation=activations,strides=1))
    for i in range(layers-1):
        model.add(Conv1D(filters=nb_filters,kernel_size=size,padding="valid",activation=activations,strides=1))
        
    model.add(MaxPooling1D(pool_size= pool_size))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
        
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    
    #y_pred = model.predict(x_test)
    #yy_test = [np.argmax(i) for i in y_test]

    #yy_pred = [np.argmax(i) for i in y_pred]
  
    #new = np.vstack([yy_test,yy_pred])
    #print(classification_report(yy_test, yy_pred, digits = 5 ))
    #f1 = f1_score(yy_test, y_pred)
    #F1Scores.append(f1)
    
    #ap = average_precision_score(yy_test, y_pred[:,1])
    #PrecisionScores.append(ap)
    

    """

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(yy_test, y_pred[:,0],pos_label=0)
    auc_keras = auc(fpr_keras, tpr_keras)
    AucScores.append(auc_keras)
    """
    del model
    gc.collect()
    K.clear_session()
    gc.collect()

    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(AucScores), np.std(AucScores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(F1Scores), np.std(F1Scores)))
#print("%.5f%% (+/- %.5f%%)" % (np.mean(PrecisionScores), np.std(PrecisionScores)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 89.10522%
loss: 0.25438%
Fit and Evaluate Time: 73.58207058906555
accuracy: 89.03645%
loss: 0.25467%
Fit and Evaluate Time: 67.9497230052948
accuracy: 89.33526%
loss: 0.25360%
Fit and Evaluate Time: 66.09202003479004
accuracy: 88.51709%
loss: 0.26865%
Fit and Evaluate Time: 66.40930938720703
accuracy: 89.10285%
loss: 0.26269%
Fit and Evaluate Time: 66.01029968261719
accuracy: 88.60246%
loss: 0.26004%
Fit and Evaluate Time: 66.26499557495117
accuracy: 89.28072%
loss: 0.25897%
Fit and Evaluate Time: 65.97678899765015
accuracy: 88.77084%
loss: 0.25462%
Fit and Evaluate Time: 65.20565915107727
accuracy: 89.34001%
loss: 0.25363%
Fit and Evaluate Time: 65.61148595809937
accuracy: 88.00958%
loss: 0.27674%
Fit and Evaluate Time: 66.53992629051208
accuracy: 89.23091%
loss: 0.26955%
Fit and Evaluate Time: 65.16773509979248
accuracy: 89.05542%
loss: 0.25409%
Fit and Evaluate Time: 65.92559599

In [4]:
AccScores

[89.105224609375,
 89.03645277023315,
 89.33526277542114,
 88.51708769798279,
 89.10285234451294,
 88.60245943069458,
 89.28071856498718,
 88.77084255218506,
 89.34000730514526,
 88.00957798957825,
 89.23091292381287,
 89.05542492866516,
 88.89890313148499,
 88.96530270576477,
 88.92261981964111,
 88.27519416809082,
 89.04356360435486,
 89.57952857017517,
 89.30443525314331,
 89.37557935714722,
 89.11945223808289,
 88.41748237609863,
 89.10759687423706,
 89.32103514671326,
 89.41826820373535,
 89.44435119628906,
 88.50522637367249,
 89.34474587440491,
 88.87755870819092,
 89.85462784767151,
 89.26411867141724,
 89.01036381721497,
 88.26807737350464,
 88.24673295021057,
 88.5431706905365,
 88.93921971321106,
 88.4388267993927,
 88.43645453453064,
 89.24751877784729,
 89.45621252059937,
 88.3866548538208,
 88.55740427970886,
 88.40088248252869,
 89.09336924552917,
 88.52419853210449,
 89.22617435455322,
 88.04515600204468,
 88.98427486419678,
 88.38902711868286]

In [7]:
LossScores

[0.2543829416274278,
 0.25466985323523983,
 0.253604208176749,
 0.26864633259369236,
 0.2626929012333001,
 0.2600378316262167,
 0.2589729799943957,
 0.2546202524696774,
 0.25363221411427456,
 0.2767420284393336,
 0.26955162904495605,
 0.25408717550116977,
 0.2582767935983965,
 0.25737923893893866,
 0.26277419243781897,
 0.2717614280874529,
 0.24872770649891732,
 0.25570197520383514,
 0.2629513985037402,
 0.261552622320169,
 0.2573881965072878,
 0.26006134653780105,
 0.2551828257172892,
 0.2559424762593509,
 0.2577970111108511,
 0.24904361947571224,
 0.2651807758127136,
 0.25775008423846124,
 0.258663061272344,
 0.24555993769185788,
 0.2507875724507777,
 0.2617237547829376,
 0.2587253046172668,
 0.26951708561742205,
 0.2620068006861523,
 0.25170213104571504,
 0.25689226990108865,
 0.26715390335801237,
 0.251633169311528,
 0.2516113939876236,
 0.2678919419576293,
 0.25702410885892296,
 0.2592890893967447,
 0.25922640642127337,
 0.26364589213584966,
 0.2529005710153328,
 0.264223068263107

In [8]:
Times

[73.58207058906555,
 67.9497230052948,
 66.09202003479004,
 66.40930938720703,
 66.01029968261719,
 66.26499557495117,
 65.97678899765015,
 65.20565915107727,
 65.61148595809937,
 66.53992629051208,
 65.16773509979248,
 65.92559599876404,
 67.15752220153809,
 65.67529726028442,
 66.60524153709412,
 66.53224110603333,
 66.14062714576721,
 65.496098279953,
 66.73258566856384,
 66.54114866256714,
 66.388028383255,
 65.91399812698364,
 66.5015778541565,
 65.91160345077515,
 65.80597567558289,
 66.72379064559937,
 65.20480751991272,
 66.84970307350159,
 65.67833209037781,
 66.23423457145691,
 65.36672830581665,
 65.6241614818573,
 66.21689987182617,
 66.76148271560669,
 65.87491464614868,
 66.12395215034485,
 65.21816635131836,
 66.76885986328125,
 65.84142899513245,
 66.20994973182678,
 65.91570496559143,
 65.62564492225647,
 65.70773124694824,
 66.72771191596985,
 65.6626398563385,
 65.69525027275085,
 65.56711387634277,
 66.36722707748413,
 65.67475295066833]